[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/02_Safety_and_Supervisor.ipynb)

# Supervisor and Safety Rules

**Goal:** understand how unsafe insulin is clamped and logged.


In [ ]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


## Step 1: Direct safety evaluation


In [ ]:
from iints.core.supervisor import IndependentSupervisor

supervisor = IndependentSupervisor()
result = supervisor.evaluate_safety(
    current_glucose=65.0,
    proposed_insulin=1.2,
    current_time=0.0,
    current_iob=0.5,
)
result


## Step 2: Safety report from a short run


In [ ]:
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


In [ ]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


In [ ]:
outputs = iints.run_simulation(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=180,
    time_step=preset["time_step_minutes"],
    seed=7,
    output_dir=None,
)

outputs["safety_report"]


### Recap
The supervisor logs reductions and safety reasons in the report.
